This is necessary because I'm using old Docker image compiled for different version: https://blog.ml6.eu/serving-decision-forests-with-tensorflow-b447ea4fc81c 

The `!` sign before commands is for Colab to run the terminal command. Disregard that in normal environment.

In [1]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.5.1 tensorflow_decision_forests==0.1.8 pandas wurlitzer

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 454.4 MB 9.0 kB/s 
     |████████████████████████████████| 6.3 MB 59.7 MB/s 
     |████████████████████████████████| 462 kB 41.2 MB/s 
     |████████████████████████████████| 4.0 MB 38.9 MB/s 
     |████████████████████████████████| 14.8 MB 47.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
     |████████████████████████████████| 132 kB 59.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68714 sha256=b42317c99357eb5fde8d8f18304cc2016545813b4cd73f442ac1d2668abdae94
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled t

In [1]:
# Download the phishing URLs file. 
# It should contain a user agent with username of phishtank, but registration is disabled...
# https://phishtank.org/developer_info.php
!wget http://data.phishtank.com/data/online-valid.csv -O /tmp/phishingUrls.csv
!wget https://raw.githubusercontent.com/shreyagopal/Phishing-Website-Detection-by-Machine-Learning-Techniques/master/DataFiles/1.Benign_list_big_final.csv -O /tmp/legitUrls.csv

--2022-03-07 15:15:39--  http://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 104.17.177.85, 104.16.101.75, 2606:4700::6811:b155, ...
Connecting to data.phishtank.com (data.phishtank.com)|104.17.177.85|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.phishtank.com/data/online-valid.csv [following]
--2022-03-07 15:15:39--  https://data.phishtank.com/data/online-valid.csv
Connecting to data.phishtank.com (data.phishtank.com)|104.17.177.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn.phishtank.com/datadumps/verified_online.csv?Expires=1646666149&Signature=G7Y0ZJdPbPPoU~piZ9RUGriXUc6rglc5P~IPZu~GdxVarNCOkbuPE9ZY7fTWXN3k6wXzewucH8J7KZQKDpiTBt2P6AvS2Ufwo8mH8L1ohd8ML26~-Fxe3lMi2IHT8t1z95jnQL8ijsWjnsQvxR2l7K-OcC7hqbYhHc3fyx8g2t4ak6AlM59iKx4Md~wH8qvnFcnIzny9DrEIwjJ3JrIF60Onms0nTCHVVa9m6poX463GuBC5uamD7uP-C4T4PQss1XEyv00x1vMaCGOezILWZikrOt8~ohJFYo-jn

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import pandas as pd

# libraries for parsing the URLs
import ipaddress
from urllib.parse import urlparse,urlencode

print('TensorFlow version: {}'.format(tf.__version__))
# should be 2.5.1

TensorFlow version: 2.5.1


In [33]:

# Helper libraries
import pandas as pd
import re

# libraries for parsing the URLs
from urllib.parse import urlparse
# https://github.com/Chandni97/PhishDetect/blob/master/extract_feature.py
# https://github.com/ESDAUNG/Phishing-URL-Detection/blob/main/Feature_extraction.java
# https://github.com/shreyagopal/Phishing-Website-Detection-by-Machine-Learning-Techniques/blob/master/URL%20Feature%20Extraction.ipynb


class UrlFeautures():

    def __init__(self, url: str):
        self.set_feautures_list(url)

    def has_ip(self, url):
        '''
        Code from https://gist.github.com/dfee/6ed3a4b05cfe7a6faf40a2102408d5d8
        slightly modified to detect IPv4 on it's own. Because it's enough to detect IPv4 on it's own, the embeded cases were deleted, as they are detected anyway.
        https://stackoverflow.com/questions/53497/regular-expression-that-matches-valid-ipv6-addresses
        '''
        IPV4SEG = r'(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])'
        IPV4ADDR = r'(?:(?:' + IPV4SEG + r'\.){3,3}' + IPV4SEG + r')'
        IPV6SEG = r'(?:(?:[0-9a-fA-F]){1,4})'
        IPV6GROUPS = (
            r'(?:' + IPV6SEG + r':){7,7}' + IPV6SEG,
            # 1::                                 1:2:3:4:5:6:7::
            r'(?:' + IPV6SEG + r':){1,7}:',
            # 1::8               1:2:3:4:5:6::8   1:2:3:4:5:6::8
            r'(?:' + IPV6SEG + r':){1,6}:' + IPV6SEG,
            # 1::7:8             1:2:3:4:5::7:8   1:2:3:4:5::8
            r'(?:' + IPV6SEG + r':){1,5}(?::' + IPV6SEG + r'){1,2}',
            # 1::6:7:8           1:2:3:4::6:7:8   1:2:3:4::8
            r'(?:' + IPV6SEG + r':){1,4}(?::' + IPV6SEG + r'){1,3}',
            # 1::5:6:7:8         1:2:3::5:6:7:8   1:2:3::8
            r'(?:' + IPV6SEG + r':){1,3}(?::' + IPV6SEG + r'){1,4}',
            # 1::4:5:6:7:8       1:2::4:5:6:7:8   1:2::8
            r'(?:' + IPV6SEG + r':){1,2}(?::' + IPV6SEG + r'){1,5}',
            # 1::3:4:5:6:7:8     1::3:4:5:6:7:8   1::8
            IPV6SEG + r':(?:(?::' + IPV6SEG + r'){1,6})',
            # ::2:3:4:5:6:7:8    ::2:3:4:5:6:7:8  ::8       ::
            r':(?:(?::' + IPV6SEG + r'){1,7}|:)',
            # fe80::7:8%eth0     fe80::7:8%1  (link-local IPv6 addresses with zone index)
            r'fe80:(?::' + IPV6SEG + r'){0,4}%[0-9a-zA-Z]{1,}',
            IPV4ADDR
        )
        # Reverse rows for greedy match
        IPV6ADDR = '|'.join(['(?:{})'.format(g) for g in IPV6GROUPS[::-1]])
        match = re.search(IPV6ADDR, url)  # Ipv6
        if match:
            return 1
        else:
            return 0

    def is_url_short(self, url):
        """Simple method that does regex search for a list of URL shortening domains
        The basic list was taken from https://github.com/Chandni97/PhishDetect/blob/master/extract_feature.py
        combined with list from https://meta.stackoverflow.com/questions/313621/blacklist-the-use-of-common-link-shorteners-in-posts
        there are likely duplicates form both lists"""
        match = re.search(
            'zi\.mu|zi\.ma|yhoo\.it|yfrog\.com|yep\.it|y\.ahoo\.it|xurl\.es|xrl\.us|'
            'xrl\.in|wp\.me|url\.ie|url\.co\.uk|url\.az|ur1\.ca|u\.nu|twurl\.nl|twurl\.cc|'
            'tr\.im|to\.ly|tnij\.org|tinyurl\.com|tinylink\.in|tiny\.pl|tiny\.ly|tiny\.cc|'
            'tcrn\.ch|ta\.gd|t\.co|t\.cn|su\.pr|sp2\.ro|snurl\.com|snipurl\.com|snipr\.com|'
            'shrt\.st|shorturl\.com|short\.ie|shorl\.com|shar\.es|sameurl\.com|safe\.mn|post\.ly|'
            'ping\.fm|ow\.ly|om\.ly|nyti\.ms|nsfw\.in|moby\.to|migre\.me|lnkd\.in|linkbun\.ch|'
            'linkbee\.com|liip\.to|krunchd\.com|korta\.nu|j\.mp|is\.gd|hurl\.me|huff\.to|goo\.gl|'
            'fwd4\.me|fff\.to|ff\.im|fb\.me|fav\.me|eepurl\.com|doiop\.com|dlvr\.it|disq\.us|'
            'digg\.com|digbig\.com|decenturl\.com|cutt\.us|cot\.ag|cli\.gs|clck\.ru|cl\.ly|'
            'chilp\.it|budurl\.com|bit\.ly|binged\.it|bacn\.me|arst\.ch|alturl\.com|afx\.cc|'
            'adjix\.com|adf\.ly|4sq\.com|3\.ly|0rz\.tw|we\.tl|ouo\.io|bfy\.tw|bit\.do|'
            'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
            'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
            'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
            'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
            'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
            'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
            'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net|shorturl\.at', url)
        if match:
            return 1
        else:
            return 0

    def get_dots_in_hostname(self, url):
        """Basic idea is that more dots in domain part = Sub Domain and Multi Sub Domains"""
        return urlparse(url).netloc.count(".")

    def has_at_sign(self, url):
        if "@" in url:
            return 1
        return 0

    # def has_double_slash(self, url):
    #    if "//" in urlparse(url).netloc:
    #        return 1
    #    return 0

    def has_double_slash(self, url):
        # since the position starts from, we have given 6 and not 7 which is according to the document
        list = [x.start(0) for x in re.finditer('//', url)]
        if list[len(list)-1] > 6:
            return 1
        else:
            return 0

    def has_hyphen_domain(self, url):
        if "-" in urlparse(url).netloc:
            return 1
        return 0

    def has_https(self, url):
        if urlparse(url).scheme == "https":
            return 1
        return 0

    def get_host_length(self, url):
        return len(urlparse(url).netloc)

    def has_hyphen_or_underscore(self, url):
        if ("_" in url or "-" in url):
            return 1
        return 0

    def get_base_url_length(self, url):
        parsedUrl = urlparse(url)
        return len(parsedUrl.scheme) + len(parsedUrl.netloc) + len(parsedUrl.path)

    def get_feautures_list(self):
        return self.feautures

    def set_feautures_list(self, url):
        self.feautures = [
            self.has_ip(url),
            self.is_url_short(url),
            self.get_dots_in_hostname(url),
            self.has_at_sign(url),
            self.has_double_slash(url),
            self.has_hyphen_domain(url),
            self.has_https(url),
            self.get_host_length(url),
            self.has_hyphen_or_underscore(url),
            self.get_base_url_length(url),
        ]

    def get_feautures_names(self):
        """Get names of all the methods in the class as a list.
        This will be used as column names in the Dataframe.
        Doesn't seem to be better way than to call __name__ on each method.
        The best would be to somehow automatically call this on all methods in the class.
        But I don't know how to do it without overcomplicaitng things."""
        return [
            self.has_ip.__name__,
            self.is_url_short.__name__,
            self.get_dots_in_hostname.__name__,
            self.has_at_sign.__name__,
            self.has_double_slash.__name__,
            self.has_hyphen_domain.__name__,
            self.has_https.__name__,
            self.get_host_length.__name__,
            self.has_hyphen_or_underscore.__name__,
            self.get_base_url_length.__name__,
        ]


class UrlFeaturesWithLabel(UrlFeautures):
    def __init__(self, url: str, label: int):
        #self.feautures = self.get_feautures_list(url)
        self.set_feautures_list(url, label)

    def set_feautures_list(self, url, label):
        """Setting the feautures, same as base class
        and adding the label."""
        super().set_feautures_list(url)
        self.feautures.append(label)

    def get_feautures_names(self):
        """Overriding feauture names and adding the label for clasificaiton"""
        feauture_names = super().get_feautures_names()
        feauture_names.append('is_phishing')
        print(feauture_names)
        return feauture_names


class DataSet:
    RANDOM_STATE = 12

    def __init__(self, legitimate_URLs: pd.DataFrame, phishing_URLs: pd.DataFrame, numberOfSamples: int):
        self.legitURLs = legitimate_URLs
        self.phishingURLs = phishing_URLs
        self.numberOfSamples = numberOfSamples
        self.data = self.createDataSet()

    def createPhishingDataFrame(self):
        phishingFeatures = []
        is_phishing = 1

        phishurl = self.phishingURLs.sample(
            n=self.numberOfSamples, random_state=self.RANDOM_STATE).copy()
        phishurl = phishurl.reset_index(drop=True)

        for i in range(0, self.numberOfSamples):
            url = phishurl['url'][i]
            feautureExtraction = UrlFeaturesWithLabel(url, is_phishing)
            phishingFeatures.append(feautureExtraction.feautures)

        return pd.DataFrame(phishingFeatures, columns=feautureExtraction.get_feautures_names())

    def createLegitimateDataFrame(self):
        legi_features = []
        is_phishing = 0

        # the data is asumed to have first column as 'url'
        self.legitURLs.columns = ['url']
        legiurl = self.legitURLs.sample(
            n=self.numberOfSamples, random_state=self.RANDOM_STATE).copy()
        legiurl = legiurl.reset_index(drop=True)

        for i in range(0, self.numberOfSamples):
            url = legiurl['url'][i]
            feautureExtraction = UrlFeaturesWithLabel(url, is_phishing)
            legi_features.append(feautureExtraction.feautures)

        return pd.DataFrame(legi_features, columns=feautureExtraction.get_feautures_names())

    def createDataSet(self):
        # concat both of the sets
        legitimate = self.createLegitimateDataFrame()
        phishing = self.createPhishingDataFrame()
        return pd.concat([legitimate, phishing]).reset_index(drop=True)


# initialising it for colab to be avilable outside of this code block
urldata = pd.DataFrame()
if (__name__ == '__main__'):
    # settings to display all columns, used for debuging
    pd.set_option("display.max_columns", None)
    legitURLs = pd.read_csv("/tmp/legitUrls.csv")
    phishingURLs = pd.read_csv("/tmp/phishingUrls.csv")
    dataSet = DataSet(legitURLs, phishingURLs, 5000)
    urldata = dataSet.data


['has_ip', 'is_url_short', 'get_dots_in_hostname', 'has_at_sign', 'has_double_slash', 'has_hyphen_domain', 'has_https', 'get_host_length', 'has_hyphen_or_underscore', 'get_base_url_length', 'is_phishing']
['has_ip', 'is_url_short', 'get_dots_in_hostname', 'has_at_sign', 'has_double_slash', 'has_hyphen_domain', 'has_https', 'get_host_length', 'has_hyphen_or_underscore', 'get_base_url_length', 'is_phishing']


In [34]:
# Split the dataset into a training and a testing dataset.
# is this choosing random indexes or do I need to shuffle them???
def split_dataset(dataset, test_ratio=0.10):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(urldata)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))


8980 examples in training, 1020 examples for testing.


Convert pandas dataframe into tensorflow dataset

In [35]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label="is_phishing")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label="is_phishing")


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:1224: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  (dict(dataframe.drop(label, 1)), dataframe[label].values))


Train the model
Note:
No input features are specified. Therefore, all the columns will be used as input features except for the label. The feature used by the model are shown in the training logs and in the model.summary()

In [36]:

# Specify the model.
model = tfdf.keras.RandomForestModel()

# Optionally, add evaluation metrics.
model.compile(
    metrics=["accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs. It doesn't work for some reason.
#with sys_pipes():
model.fit(x=train_ds)

  


141/141 [==============================] - 0s 1ms/step


In [37]:
evaluation = model.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")


16/16 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.9706

loss: 0.0000
accuracy: 0.9706


In [8]:
#model.summary()
#model.make_inspector().variable_importances()
#model.make_inspector().evaluation()
model.make_inspector().features()



["base_url_length" (1; #0),
 "has_at_sign" (1; #1),
 "has_https" (1; #2),
 "has_hyphen_or_underscore" (1; #3),
 "host_name_length" (1; #4),
 "number_of_dots_in_hostname" (1; #5)]

In [38]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)


Default TF-serving produces error. Finally found why that's happening. The explanation:
>TensorFlow Serving is a serving system for TensorFlow models in production environments. The TF-Serving team publishes a pre-compiled release containing only Core TensorFlow ops.

>TensorFlow Decision Forests (TF-DF) models use custom Ops for inference. Therefore, they are not compatible with the pre-compiled releases of TF-Serving. The solution to serve TF-DF models with TF-Serving is to re-compile TF-Serving with TF-DF ops. This document explains how to do so.

https://github.com/tensorflow/decision-forests/blob/main/documentation/tensorflow_serving.md

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [40]:
MODEL_DIR = "/content/gdrive/MyDrive/phishingModelMoreFeautures"
# model_1.save("/content/gdrive/MyDrive/firstModelSave")
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}


export_path = /content/gdrive/MyDrive/phishingModelMoreFeautures/1

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/phishingModelMoreFeautures/1/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/phishingModelMoreFeautures/1/assets



Saved model:
total 152
drwx------ 2 root root   4096 Mar  7 15:51 assets
-rw------- 1 root root   2987 Mar  7 15:51 keras_metadata.pb
-rw------- 1 root root 144216 Mar  7 15:51 saved_model.pb
drwx------ 2 root root   4096 Mar  7 15:51 variables
